In [1]:
import pandas as pd
import numpy as np
import os
import ast
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from shutil import copyfile
import yaml

In [2]:
# Install W&B 
!pip install -q --upgrade wandb

# Login 
import wandb
print(wandb.__version__)
wandb.login(anonymous='must')

0.12.7


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
df = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
df.head()

,video_id,sequence,video_frame,sequence_frame,image_id,annotations
0,0,40258,0,0,0-0,[]
1,0,40258,1,1,0-1,[]
2,0,40258,2,2,0-2,[]
3,0,40258,3,3,0-3,[]
4,0,40258,4,4,0-4,[]


In [4]:
#finding no. of samples without any bounding boxes
df.annotations = df.annotations.apply(ast.literal_eval)
df['no_of_bboxes'] = df['annotations'].apply(lambda x: len(x))

# drop rows with 0 bounding boxes
df = df.loc[lambda df: df['no_of_bboxes']>0]
df.reset_index(inplace=True)
df = df.drop(['index'], axis=1)
df.head()

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,no_of_bboxes
0,0,40258,16,16,0-16,"[{'x': 559, 'y': 213, 'width': 50, 'height': 32}]",1
1,0,40258,17,17,0-17,"[{'x': 558, 'y': 213, 'width': 50, 'height': 32}]",1
2,0,40258,18,18,0-18,"[{'x': 557, 'y': 213, 'width': 50, 'height': 32}]",1
3,0,40258,19,19,0-19,"[{'x': 556, 'y': 214, 'width': 50, 'height': 32}]",1
4,0,40258,20,20,0-20,"[{'x': 555, 'y': 214, 'width': 50, 'height': 32}]",1


In [5]:
print(df.no_of_bboxes.values.max())

18


In [6]:
def retrieve_bounding_box(list_of_annotations):
    '''
    Retrieve the bounding box values for training YOLO
    '''
    bboxes = []
    for bbox in list_of_annotations:
        width = bbox['width']
        height = bbox['height']
        xc = bbox['x'] + int(width/2)
        yc = bbox['x'] + int(height/2)
        xc/=1280
        width/=1280
        yc/=720
        height/=720
        modified_bbox = [0, xc, yc, width, height]
        bboxes.append(modified_bbox)
        
        #we did this because YOLO expects data in a particular format as shown below:
        # (x_centre, y_centre, box_width, box_height)
    return bboxes

In [7]:
df['bbox'] = [retrieve_bounding_box(df.annotations[i]) for i in range(len(df))]
df['image_path'] = None
for i in range(len(df)):
    path = f"../input/tensorflow-great-barrier-reef/train_images/video_{df.video_id[i]}/{df.video_frame[i]}.jpg"
    df.image_path[i] = path
df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,video_id,sequence,video_frame,sequence_frame,image_id,annotations,no_of_bboxes,bbox,image_path
0,0,40258,16,16,0-16,"[{'x': 559, 'y': 213, 'width': 50, 'height': 32}]",1,"[[0, 0.45625, 0.7986111111111112, 0.0390625, 0...",../input/tensorflow-great-barrier-reef/train_i...
1,0,40258,17,17,0-17,"[{'x': 558, 'y': 213, 'width': 50, 'height': 32}]",1,"[[0, 0.45546875, 0.7972222222222223, 0.0390625...",../input/tensorflow-great-barrier-reef/train_i...
2,0,40258,18,18,0-18,"[{'x': 557, 'y': 213, 'width': 50, 'height': 32}]",1,"[[0, 0.4546875, 0.7958333333333333, 0.0390625,...",../input/tensorflow-great-barrier-reef/train_i...
3,0,40258,19,19,0-19,"[{'x': 556, 'y': 214, 'width': 50, 'height': 32}]",1,"[[0, 0.45390625, 0.7944444444444444, 0.0390625...",../input/tensorflow-great-barrier-reef/train_i...
4,0,40258,20,20,0-20,"[{'x': 555, 'y': 214, 'width': 50, 'height': 32}]",1,"[[0, 0.453125, 0.7930555555555555, 0.0390625, ...",../input/tensorflow-great-barrier-reef/train_i...


In [8]:
train_df, valid_df = train_test_split(
    df,
    test_size = 0.1,
    random_state = 42,
    shuffle = True
)

train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)

In [9]:
# Execute the following set of commands just once
%pwd
!mkdir the_great_barrier_reef_data
%cd the_great_barrier_reef_data
!mkdir images
!mkdir labels
%cd images
!mkdir train
!mkdir validation
%cd ..
%cd labels
!mkdir train
!mkdir validation
%cd ..
%cd ..
!tree

/kaggle/working/the_great_barrier_reef_data
/kaggle/working/the_great_barrier_reef_data/images
/kaggle/working/the_great_barrier_reef_data
/kaggle/working/the_great_barrier_reef_data/labels
/kaggle/working/the_great_barrier_reef_data
/kaggle/working
.
├── __notebook__.ipynb
└── the_great_barrier_reef_data
    ├── images
    │   ├── train
    │   └── validation
    └── labels
        ├── train
        └── validation

7 directories, 1 file


In [10]:
for i in tqdm(range(len(train_df))):
    row = train_df.loc[i]
    copyfile(f'{row.image_path}', f'./the_great_barrier_reef_data/images/train/{row.image_id}.jpg')

for i in tqdm(range(len(valid_df))):
    row = valid_df.loc[i]
    copyfile(f'{row.image_path}', f'./the_great_barrier_reef_data/images/validation/{row.image_id}.jpg')    

100%|██████████| 492/492 [00:05<00:00, 85.23it/s]


In [11]:
for i in tqdm(range(train_df.shape[0])):
    row = train_df.loc[i]
    yolo_label_data = row.bbox
    yolo_label_data = np.array(yolo_label_data)
    np.savetxt(
        f"./the_great_barrier_reef_data/labels/train/{row.image_id}.txt",
        yolo_label_data,
        fmt=["%d", "%f", "%f", "%f", "%f"]
    )

for i in tqdm(range(valid_df.shape[0])):
    row = valid_df.loc[i]
    yolo_label_data = row.bbox
    yolo_label_data = np.array(yolo_label_data)
    np.savetxt(
        f"./the_great_barrier_reef_data/labels/validation/{row.image_id}.txt",
        yolo_label_data,
        fmt=["%d", "%f", "%f", "%f", "%f"]
    )

100%|██████████| 492/492 [00:00<00:00, 782.72it/s]


In [12]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 10177, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 10177 (delta 3), reused 8 (delta 3), pack-reused 10169
Receiving objects: 100% (10177/10177), 10.41 MiB | 14.83 MiB/s, done.
Resolving deltas: 100% (7055/7055), done.


In [13]:
%ls yolov5/

CONTRIBUTING.md  README.md  export.py   requirements.txt*  tutorial.ipynb
Dockerfile       data/      hubconf.py  setup.cfg          utils/
LICENSE          detect.py  models/     train.py           val.py


In [14]:
data = dict(train = '/kaggle/working/the_great_barrier_reef_data/images/train',
        val = '/kaggle/working/the_great_barrier_reef_data/images/validation',
        nc = 1,
        names = ['COTS']
        )

with open('/kaggle/working/yolov5/data/data.yaml', 'w') as outfile:
    yaml.dump(data, outfile)

%cat /kaggle/working/yolov5/data/data.yaml

names:
- COTS
nc: 1
train: /kaggle/working/the_great_barrier_reef_data/images/train
val: /kaggle/working/the_great_barrier_reef_data/images/validation


In [15]:
%cp /kaggle/input/pretrained-weights-yolov5s/yolov5s.pt /kaggle/working/yolov5/

In [16]:
%cd yolov5/

/kaggle/working/yolov5


In [17]:
!python train.py --img 1280 --batch 16 --epochs 30 --data data.yaml --cfg models/yolov5s.yaml --name finetuned_YOLO5s_with_35_epochs --weights yolov5s.pt

wandb: Currently logged in as: anony-mouse-170823 (use `wandb login --relogin` to force relogin)
github: up to date with https://github.com/ultralytics/yolov5 ✅
wandb: Tracking run with wandb version 0.12.7
wandb: Syncing run finetuned_YOLO5s_with_35_epochs
wandb: ⭐️ View project at https://wandb.ai/anony-mouse-170823/YOLOv5?apiKey=46bce5cb786fe3562c5c269dc78994d11a6a4d44
wandb: 🚀 View run at https://wandb.ai/anony-mouse-170823/YOLOv5/runs/1ympoeke?apiKey=46bce5cb786fe3562c5c269dc78994d11a6a4d44
wandb: Run data is saved locally in /kaggle/working/yolov5/wandb/run-20211211_130042-1ympoeke
wandb: Run `wandb offline` to turn off syncing.
wandb: WARNING Do NOT share these links with anyone. They can be used to claim your runs.




































































wandb: Waiting for W&B process to finish, PID 188... (success).
wandb:                                                                                
wandb: Run history:
wandb:        metrics/mAP_0.

In [18]:
%cd ..

/kaggle/working


In [19]:
import shutil
shutil.make_archive('yolov5','zip','/kaggle/working/yolov5/')
from IPython.display import FileLink
FileLink(r'yolov5.zip')

/kaggle/working/yolov5.zip